In [8]:
import random
import numpy as np  
import pandas as pd
from keras.preprocessing import sequence
from keras.preprocessing import text
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, CuDNNLSTM, LSTM, Dense, Bidirectional, BatchNormalization, Dropout, Reshape, Concatenate, Add
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras import backend as K
from keras import optimizers
from keras.utils import to_categorical
import time
import datetime

In [2]:
# Generate a dataset of strings comprising either odd or even integers, and their corresponding integer labels (1 or 2)

def generate_numstring(seed):
    numbers = [random.randrange(seed, 1000, 2) for x in range(random.randint(3, 7))]
    numbers.sort()
    return " ".join([str(x) for x in numbers])

def build_dataset(n_samples):
    n_samples = n_samples // 2
    evens_list = [generate_numstring(0) for x in range(n_samples)]
    evens_labels = [2 for x in range(n_samples)]
    odds_list = [generate_numstring(1) for x in range(n_samples)]
    odds_labels = [1 for x in range(n_samples)]
    strings_pre = evens_list + odds_list
    labels_pre = evens_labels + odds_labels
    merge = list(zip(strings_pre, labels_pre))
    random.shuffle(merge)
    strings, labels = zip(*merge)
    return(strings, labels)

data = build_dataset(100000)
X = data[0]
X2 = ["<s> "+x+" <e>" for x in X]
X3 = [x+" <e>" for x in X]
y = data[1]

for row in range(10):
    print("{} = {}".format(X[row], y[row]))

59 285 609 = 1
94 252 302 420 494 820 = 2
47 93 527 635 665 695 879 = 1
67 593 791 959 969 975 = 1
143 421 775 947 = 1
68 80 574 790 = 2
68 212 452 656 762 764 798 = 2
32 208 306 590 916 = 2
219 369 567 769 = 1
244 326 638 = 2


In [3]:
# Set Parameters
training_ratio = .75
training_size = int(len(X)*training_ratio)
num_classes = 3
embedding_vector_length = 200
H = 500
epochs = 100
optimizer = 'rmsprop'
batch_size = 64
learning_rate = .0001

# Encode strings
t = text.Tokenizer(num_words=5000, lower=True, char_level=False, filters='')

# Convert strings to sequences, pad them to uniform length, and divide up training and test sets
t.fit_on_texts(X2)
index_word = {v: k for k, v in t.word_index.items()}
V = len(t.word_index) + 1
X_seq = t.texts_to_sequences(X)
X2_seq = t.texts_to_sequences(X2)
X3_seq = t.texts_to_sequences(X3)
x_length = max(len(x) for x in X2_seq)
X_padded = sequence.pad_sequences(X_seq, maxlen=x_length, padding='post')
X2_padded = sequence.pad_sequences(X2_seq, maxlen=x_length, padding='post')
X3_padded = sequence.pad_sequences(X3_seq, maxlen=x_length, padding='post')

X_train = X_padded[:training_size]
X2_train = X2_padded[:training_size]
X3_train = X3_padded[:training_size]
X_test = X_padded[training_size:]
X2_test = X2_padded[training_size:]
y_train = y[:training_size]
y_test = y[training_size:]

# One-hot encode labels
encoded_y_train = to_categorical(y_train, num_classes=num_classes)
encoded_y_test = to_categorical(y_test, num_classes=num_classes)

X_train_target = to_categorical(X3_train, num_classes=V)

In [9]:
# define training encoder
encoder_inputs = Input(shape=(None, ), name="encoder_input")
encoder = Embedding(V, embedding_vector_length, name="encoder_embedding")(encoder_inputs)
encoder = Bidirectional(CuDNNLSTM(H, return_sequences=True), name="encoder_lstm_1")(encoder)
encoder_outputs, forward_h, forward_c, backward_h, backward_c = Bidirectional(CuDNNLSTM(H, return_state=True), name="encoder_lstm_2")(encoder)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_normalizer = BatchNormalization()
encoder_dense = Dense(num_classes, activation='softmax', name="encoder_dense")
encoder_outputs = encoder_normalizer(state_h)
encoder_outputs = encoder_dense(encoder_outputs)
encoder_states = [state_h, state_c]

# define training decoder
decoder_inputs = Input(shape=(None, ), name="decoder_input")
decoder_embedding = Embedding(V, embedding_vector_length)
embedded_input = decoder_embedding(decoder_inputs)
decoder_lstm = CuDNNLSTM(H*2, return_sequences=True, return_state=True, name="decoder_lstm_2")
decoder_outputs, _, _ = decoder_lstm(embedded_input, initial_state=encoder_states)
decoder_normalizer = BatchNormalization()
decoder_dense = Dense(V, activation='softmax', name="decoder_dense")
decoder_outputs = decoder_normalizer(decoder_outputs)
decoder_outputs = decoder_dense(decoder_outputs)

# Combine training inputs into a single training model
model = Model([encoder_inputs, decoder_inputs], [encoder_outputs, decoder_outputs])

# define inference encoder
encoder_model = Model(encoder_inputs, [encoder_outputs] + encoder_states)

# define inference decoder
decoder_state_input_h = Input(shape=(H*2,))
decoder_state_input_c = Input(shape=(H*2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_input_2 = decoder_embedding(decoder_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_input_2, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [10]:
start_time = time.time()

# Compile and train the model
opt = optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
model.summary()
callbacks = [EarlyStopping(monitor='loss', patience=2, min_delta=0.05, restore_best_weights=True)]
            # TensorBoard(log_dir='./logs/PoC_Word/1000_Words_1Lx1L_mask', histogram_freq=0, batch_size=32, write_graph=False, 
            #             write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, 
            #             embeddings_metadata=None, embeddings_data=None, update_freq='epoch')]
model.fit([X_train, X2_train], [encoded_y_train, X_train_target], callbacks=callbacks, 
          epochs=1, batch_size=batch_size, validation_split=0.1)

# Final evaluation of the model
end_time = time.time()
run_time = datetime.timedelta(seconds=end_time-start_time)
print("Finished in {}".format(run_time))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
encoder_embedding (Embedding)   (None, None, 200)    200600      encoder_input[0][0]              
__________________________________________________________________________________________________
encoder_lstm_1 (Bidirectional)  (None, None, 1000)   2808000     encoder_embedding[0][0]          
__________________________________________________________________________________________________
encoder_lstm_2 (Bidirectional)  [(None, 1000), (None 6008000     encoder_lstm_1[0][0]             
__________________________________________________________________________________________________
decoder_in

OSError: Unable to create file (unable to open file: name = '/var/models/PoC_100k_2L.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)